In [1]:
import glob
from asp_plot.icesat2 import Icesat2

In [2]:
dem_fn = glob.glob("/Users/ben/Dropbox/UW_Shean/WV/2022/WV03_20220417_1040010074793300_1040010075633C00/stereo*/*DEM_1m.tif")[0]

geojson_fn = "/Users/ben/Dropbox/UW_Shean/WV/2022/utqiagvik_wv_2022_sliderule_poly.geojson"

In [3]:
icesat2 = Icesat2(dem_fn=dem_fn, geojson_fn=geojson_fn)

In [4]:
icesat2.pull_sliderule_data()

[{'lon': -156.27875181715967, 'lat': 71.23928395454953},
 {'lon': -156.25426170470448, 'lat': 71.41276421658509},
 {'lon': -156.9626248894057, 'lat': 71.42170678237126},
 {'lon': -156.98080923231944, 'lat': 71.24813787781376},
 {'lon': -156.27875181715967, 'lat': 71.23928395454953}]